In [ ]:
import pandas as pd
from ipyleaflet import Map, Heatmap, basemaps, FullScreenControl
import ipyleaflet as leaf
import ipywidgets as wid
import bqplot as bq

In [ ]:
class Wells(wid.VBox):
    def __init__(self, title='Development Wells in Norway'):
        super().__init__()
        self.df = pd.read_csv(
            './data/wellbore_development.csv'
        ).drop_duplicates(["wlbNsDecDeg", "wlbEwDesDeg"]).reset_index(drop=True)
        self.map = self.get_map()
        self.children = [
            wid.HTML(f'<h1>{title}</h1>'),
            self.map,
        ]
        
    def get_map(self):
        heatmap = leaf.Heatmap(
            locations=[(r.wlbNsDecDeg, r.wlbEwDesDeg) for idx, r in self.df.iterrows()],
            radius=20, min_opacity=0.2
        )
        m = leaf.Map(center=(65, 0), zoom=3, scroll_wheel_zoom=True)
        m.add_layer(heatmap);
        m.add_control(leaf.FullScreenControl())
        return m

In [ ]:
class Sensor(wid.VBox):
    def __init__(self, title='Vessel Data'):        
        super().__init__()
        self.df = pd.read_csv(
            './data/vessel_dof.csv', index_col=0, parse_dates=True
        ).dropna()
        self.var_select = wid.Dropdown(options=self.df.columns, description='Variable')
        self.var_select.observe(self.var_change, names='value')
        self.start_select = wid.DatePicker(
            description='Start date',
            disabled=False, value=self.df.index.max()-pd.Timedelta(weeks=1)
        )
        self.end_select = wid.DatePicker(
            description='End date',
            disabled=False, value=self.df.index.max()
        )
        self.figure = self.get_figure()
        wid.dlink((self.start_select, "value"), (self.figure["ax"].scale, "min"))
        self.children = [
            wid.HTML(f'<h1>{title}</h1>'),
            self.var_select,
            self.start_select,
            self.end_select,
            self.figure["fig"]
        ]
        
    def var_change(self, change):
        var = change["new"]
        self.figure["line"].y = self.df[var]
        
    def get_figure(self):
        df = self.df
        var = self.var_select.value
        x_sc = bq.DateScale(min=self.start_select.value, max=self.end_select.value)
        y_sc = bq.LinearScale()

        line = bq.Lines(
            x = [pd.Timestamp(i) for i in df.index],
            y = df[var],
            scales={'x': x_sc, 'y':y_sc},
        )
        
        ax_x = bq.Axis(
            scale=x_sc, 
            grid_lines='solid',
        )
        ax_y = bq.Axis(
            scale=y_sc, 
            orientation='vertical', 
            grid_lines='solid', 
        )
        f = bq.Figure(
            marks=[line], 
            axes=[ax_x, ax_y], 
            layout = wid.Layout(width='auto', height='700px'),
            interaction=bq.interacts.PanZoom(
                scales={'x':[x_sc], 'y':[y_sc]}
            )
        )
        return {
            'fig': f,
            'line': line,
            'ax': ax_x,
            'ay': ax_y
        }

In [ ]:
class App(wid.GridBox):
    def __init__(self):
        super().__init__()
        self.layout = wid.Layout(
            width='100%',
            height='100%',
            grid_template_columns='auto auto',
            grid_template_rows='auto',
            grid_gap='5px'
       )        
        self.map = Wells()
        self.plot = Sensor()
        self.children = [
            self.map,
            self.plot
        ]
app = App()
app